In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

기사 개별 url 가져오기

In [18]:
url = 'https://search.daum.net/search'

search_keywords = ['도시형 스마트팜', '도심형 스마트팜', '수직형 스마트팜']

params = {
    'w': 'news',
    'nil_search': 'btn',
    'DA': 'PGD',
    'enc': 'utf8',
    'cluster': 'y',
    'cluster_page': '1',
    'q': '수직형 스마트팜',  # 수직형 스마트팜, 도심형 스마트팜, 도시형 스마트팜
    'show_dns': '1',
    'sd': '20180101000000',  # 뉴스 검색 시작 날짜
    'ed': '20230930235959',  # 뉴스 검색 마지막 날짜 20230930235959
    'period': 'u',
    'p': 1,  # 페이지 1부터 같은 페이지가 나올 때까지 while True로 구현 += 1
    'sort': 'old',
}

# 기사의 개별 url 가져오는 함수
def get_urls(soup):
    urls = []
    for item in soup.select('.item-title'):
        urls.append(item.select_one('a').attrs['href'])

    return urls


# 한 화면에 기사 20개가 담긴 페이지에서, 원하는 기간 동안, 기사 개별 url 가져오기 (페이지 넘김 기능)
result = []
previous_urls = None
while True:
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    urls = get_urls(soup)  
    # current_page_content = str(soup)

    if previous_urls == urls:
        break

    # time.sleep(1)
    print(f"Page {params['p']} URLs:", urls)

    params['p'] += 1
    previous_urls = urls
    result.extend(urls)
print(len(result))

Page 1 URLs: ['http://v.daum.net/v/20180330170503197', 'http://v.daum.net/v/20180416110043949', 'http://v.daum.net/v/20180417030019975', 'http://v.daum.net/v/20180423060235509', 'http://v.daum.net/v/20180623072013344', 'http://v.daum.net/v/20180723060224702', 'http://v.daum.net/v/20190305170417398', 'http://v.daum.net/v/20190312114200478', 'http://v.daum.net/v/20190829090614617', 'http://v.daum.net/v/20190829090736703']
Page 2 URLs: ['http://v.daum.net/v/20190920144203728', 'http://v.daum.net/v/20190923030340663', 'http://v.daum.net/v/20190926172818580', 'http://v.daum.net/v/20200211115116657', 'http://v.daum.net/v/20200305170134985', 'http://v.daum.net/v/20200310152504856', 'http://v.daum.net/v/20200318170129587', 'http://v.daum.net/v/20200408143924066', 'http://v.daum.net/v/20200408145712995', 'http://v.daum.net/v/20200515165710346']
Page 3 URLs: ['http://v.daum.net/v/20200515183808050', 'http://v.daum.net/v/20200516091009358', 'http://v.daum.net/v/20200518155400041', 'http://v.daum.

다음 뉴스 본문 가져오기

In [19]:
def get_articles_to_df(result):
    articles = []
    for url in result:
        article = {}
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.select_one('#mArticle > div.head_view > h3').text
        date = ''.join([data.zfill(2) for data in re.findall(r'\d+', soup.select_one('.info_view').text)[:3]])
        # date = soup.select_one('#mArticle > div.head_view > div.info_view > span:nth-child(2)').text
        content = re.sub('\n|\t|\r','', soup.select_one('.article_view').text.strip())

        article = {
            'title' : title,
            'date' : date,
            'content' : content
        }
        articles.append(article)
    df = pd.DataFrame(articles)
    return df
    
df = get_articles_to_df(result)

In [20]:
df

,title,date,content
0,의령군 식물공장 공모사업 선정,20180330,경남 의령군(군수 오영호)이 농식품부에서 주최한 2018년도 수직형농장 비즈니스 모...
1,"농식품부, 한국형 '프리바' 육성 모색..농업계 기업 반감 해소 과제",20180416,김현수 농림축산식품부 차관이 지난달 24일 경북 상주시 ‘새봄 농업회사법인’ 스마트...
2,"""2022년까지 전국 네 곳""..스마트팜 거점 밑그림 나왔다(종합)",20180417,김현수 농림축산식품부 차관이 지난달 24일 경북 상주시 ‘새봄 농업회사법인’ 스마트...
3,"농식품부, 스마트팜 혁신밸리 후보지 공모 개시",20180423,김현수 농림축산식품부 차관이 지난달 24일 경북 상주시 ‘새봄 농업회사법인’ 스마트...
4,"농정원, 26일 수직형 농장 실증사업 참여업체 설명회",20180623,[세종=이데일리 이진철 기자] 농림수산식품교육문화정보원(농정원)은 오는 26일 오후...
...,...,...,...
175,"그린케이팜, 농업분야 온실가스 감축사업 지원 농가 1호 선정",20230920,[아이뉴스24 고종민 기자] 스마트팜 전문 기업 그린플러스의 자회사 그린케이팜이 농...
176,"그린케이팜, 온실가스 감축사업 지원 대상 농가 1호 선정",20230920,"(왼쪽부터) 한국농업기술진흥원 안호근 원장, 농림축산식품부 강형석 기획조정실장, N..."
177,"그린케이팜, ‘온실가스 감축사업’ 농가 1호로 선정",20230920,[이데일리 최훈길 기자] 스마트팜 전문기업 그린플러스(186230)가 친환경 탄소중...
178,"'딸기 식물공장'으로 채산성 높여..엘앤피, 경북도지사상",20230920,박은호 엘앤피 대표가 우수스타트업에 선정돼 경북도지사상을 수상한 뒤 기념 촬영 중이...


In [21]:
df.to_csv(f"articles_{params['q']}_{params['sd'][:8]}_{len(result)}.csv", index=None)

---
json dump 필요시

In [22]:
# list -> json 파일로 내보내기
import json

file_path = f"urls_{params['q']}_{params['sd'][:8]}_{len(result)}.json"

# Write the list to a JSON file
with open(file_path, "w") as json_file:
    json.dump(result, json_file)

print(f"list -> json 파일로 내보내기 완료 {file_path}")

list -> json 파일로 내보내기 완료 urls_수직형 스마트팜_20180101_180.json


In [ ]:
# json 파일 -> list 로 읽기
with open(file_path, "r") as json_file:
    result= json.load(json_file)
print(result)

---

- csv 파일 merge
- duplicate 제거
- csv 파일로 내보내기

In [36]:
import os
os.listdir()

['articles_도시형 스마트팜_20180101_327.csv',
 'articles_도심형 스마트팜_20180101_265.csv',
 'articles_수직형 스마트팜_20180101_180.csv',
 'crawl_daum_url_by_search_test_1011.ipynb',
 'crawl_daum_url_by_search_test_1012.ipynb',
 'urls_도시형 스마트팜_20180101_327.json',
 'urls_도심형 스마트팜_20180101_265.json',
 'urls_수직형 스마트팜_20180101_180.json']

In [35]:
total_df=pd.DataFrame()
for file in os.listdir()[:3]:
    temp= pd.read_csv(file, index_col=0)
    total_df=pd.concat([total_df, temp], axis=0)
total_df

,title,date,content
0,전남 '솔라시도' 에 스마트 시티 본격 조성 기대감,20180111,문재인 대통령이 2018년 신년사에서 스마트 시티 조성을 언급함에 따라 전남 해남 ...
1,"""미래 농업경쟁력 핵심은 빅데이터""",20180122,"【 앵커멘트 】 4차 산업혁명 시대, 농업도 예외가 아니죠. 과거엔 농업하면 시설이..."
2,"""미래 농업경쟁력 핵심은 빅데이터""",20180123,"【 앵커멘트 】 4차 산업혁명 시대, 농업도 예외가 아니죠. 과거엔 농업하면 시설이..."
3,[스마트시티 도래②]미래투자vs역할제한..정부·업계 '시각차',20180529,김현미 국토교통부 장관이 25일 서울 수서역 인근 스마트시티 홍보관 '더 스마티움'...
4,경북도 '스마트시티 경북현장적용 모델개발' 용역 최종 보고회,20180725,[이임태 기자(=안동)] 경상북도는 지난 24일 ICT융복합 기술을 활용하여 도시문...
...,...,...,...
175,"그린케이팜, 농업분야 온실가스 감축사업 지원 농가 1호 선정",20230920,[아이뉴스24 고종민 기자] 스마트팜 전문 기업 그린플러스의 자회사 그린케이팜이 농...
176,"그린케이팜, 온실가스 감축사업 지원 대상 농가 1호 선정",20230920,"(왼쪽부터) 한국농업기술진흥원 안호근 원장, 농림축산식품부 강형석 기획조정실장, N..."
177,"그린케이팜, ‘온실가스 감축사업’ 농가 1호로 선정",20230920,[이데일리 최훈길 기자] 스마트팜 전문기업 그린플러스(186230)가 친환경 탄소중...
178,"'딸기 식물공장'으로 채산성 높여..엘앤피, 경북도지사상",20230920,박은호 엘앤피 대표가 우수스타트업에 선정돼 경북도지사상을 수상한 뒤 기념 촬영 중이...


In [41]:
total_df.drop_duplicates(inplace=True)
total_df.to_csv(f"articles_total_{params['sd'][:8]}_{len(total_df)}.csv", index=None)

---
참고
- 리팩토링하면서 삭제한 코드

In [ ]:
# 날짜 형식 바꾸기 함수 생성
def modify_date(row):
    tmp = row.split('.')
    year = tmp[0]
    if len(tmp[1]) == 1:
        month = '0' + tmp[1]
    else:
        month = tmp[1]
    if len(tmp[2]) == 1:
        day = '0' + tmp[2]
    else:
        day = tmp[2]
    date = year + month + day
    return date

df['date'] = df['date'].apply(modify_date)

"""
위 함수 생성해서 apply 적용할 필요 없이 기사 날짜 가져올 때, zfill로 해결 가능함!
''.join([data.zfill(2) for data in re.findall(r'\d+', soup.select_one('.info_view').text)[:3]])
"""